# chapter 5

## ch5.1

### 학습 모델 저장하고 재사용하기

In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

Instructions for updating:
non-resource variables are not supported in the long term


#### 데이터 파일을 읽어 변환, 1열과 2열은 x_data, 3열부터 마지막 열까지는 y_data

In [2]:
data = np.loadtxt('./data.csv', delimiter=',', unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#### global_step 은 학습에 직접 사용되지는 않고, 학습횟수를 카운트하는 변수이다. (trainable=False)

In [3]:
global_step = tf.Variable(0,trainable=False, name='global_step')

In [4]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [5]:
W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X,W1))

In [6]:
W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1,W2))

In [7]:
W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.))
model = tf.matmul(L2,W3)

In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

#### 신경망 모델 학습

In [10]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [11]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess,ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [12]:
for step in range(2):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' %sess.run(global_step),
         'Cost: %.3f' %sess.run(cost,feed_dict={X: x_data, Y: y_data}))

Step: 1,  Cost: 0.763
Step: 2,  Cost: 0.737


In [13]:
saver.save(sess, './model/dnn/ckpt', global_step=global_step)

'./model/dnn/ckpt-2'

In [14]:
prediction = tf.argmax(model,1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


### 텐서보드 사용하기

In [31]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

In [32]:
data = np.loadtxt('./data.csv', delimiter=',',unpack=True, dtype='float32')

In [33]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

#### 신경망 모델 구성

In [34]:
global_step = tf.Variable(0, trainable=False, name='global_step')

In [35]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [36]:
with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X,W1))

In [37]:
with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1,W2))

In [38]:
with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

In [39]:
with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

In [40]:
tf.summary.scalar('cost',cost)

<tf.Tensor 'cost_1:0' shape=() dtype=string>

#### 신경망 모델 학습

In [41]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [42]:
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [43]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step %d, '%sess.run(global_step),
         'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
    
    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

Step 1,  Cost: 0.998


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float
	 [[node Placeholder_2 (defined at c:\users\jiyun lee\appdata\local\programs\python\python37\lib\site-packages\tensorflow_core\python\framework\ops.py:1751) ]]

Original stack trace for 'Placeholder_2':
  File "c:\users\jiyun lee\appdata\local\programs\python\python37\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\jiyun lee\appdata\local\programs\python\python37\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\jiyun lee\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\jiyun lee\appdata\local\programs\python\python37\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "c:\users\jiyun lee\appdata\local\programs\python\python37\lib\site-packages\ipykernel\kernelapp.

In [ ]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

In [ ]:
prediction = tf.argmax(model,1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction,feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' %sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))